<a href="https://colab.research.google.com/github/prxgyzzz/pluggg/blob/main/berttt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch scikit-learn seqeval numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sh

In [4]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=88fd09ebdbb54bf138764f62ab0fd010356ee56754e531b9ae73e7eb64d5a70f
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [8]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 67.0 MB/s eta 0:00:00


In [6]:
import json

# Definição dos seus rótulos NER
# 'O' = Outside (não é uma entidade)
# 'B-PARAMETRO' = Começo de uma entidade de PARÂMETRO
# 'I-PARAMETRO' = Interior de uma entidade de PARÂMETRO
# 'B-QUALIDADE' = Começo de uma entidade de QUALIDADE/SENTIMENTO
# 'I-QUALIDADE' = Interior de uma entidade de QUALIDADE/SENTIMENTO
raw_labels = ["O", "B-PARAMETRO", "I-PARAMETRO", "B-QUALIDADE", "I-QUALIDADE"]
label_to_id = {label: i for i, label in enumerate(raw_labels)}
id_to_label = {i: label for i, label in enumerate(raw_labels)}

# Dataset de exemplo com mais frases e variedade
# Cada entrada tem 'tokens' (palavras da frase) e 'ner_tags' (rótulos correspondentes)
data = {
    "tokens": [
        ["Coloque", "um", "reverb", "bem", "espacial", "na", "voz", "."],
        ["O", "delay", "deveria", "ser", "curto", "e", "pulsante", "."],
        ["Quero", "um", "timbre", "de", "sintetizador", "quente", "e", "analógico", "."],
        ["Ajuste", "o", "cutoff", "para", "um", "som", "mais", "escuro", "."],
        ["Dê", "um", "boost", "no", "grave", "com", "o", "EQ", "para", "uma", "sensação", "poderosa", "."],
        ["Adicione", "mais", "sustain", "e", "decay", "para", "algo", "mais", "fluido", "."],
        ["Preciso", "de", "um", "compressores", "agressivo", "na", "bateria", "."],
        ["Faça", "o", "piano", "soar", "brilhante", "e", "limpo", "."],
        ["Use", "um", "delay", "com", "muito", "feedback", "para", "um", "ambiente", "psicodélico", "."],
        ["O", "attack", "precisa", "ser", "rápido", "e", "o", "release", "curto", "."],
        ["Mude", "a", "ressonância", "para", "criar", "um", "som", "mais", "aberto", "."],
        ["A", "mixagem", "está", "muito", "seca", ",", "adicione", "um", "pouco", "de", "reverb", "."],
        ["Gostaria", "de", "um", "som", "de", "guitarra", "distorcido", "e", "pesado", "."],
        ["Deixe", "o", "som", "do", "baixo", "mais", "denso", "e", "cheio", "."],
        ["Reduza", "o", "volume", "do", "kick", "e", "aumente", "o", "snare", "para", "uma", "batida", "mais", "definida", "."],
        ["Um", "tremolo", "com", "rate", "lento", "daria", "uma", "sensação", "calma", "."],
        ["O", "filtro", "passa-baixa", "deve", "deixar", "o", "timbre", "escuro", "e", "grosso", "."],
        ["Aumente", "o", "pan", "do", "backing", "vocal", "para", "um", "som", "mais", "amplo", "."],
        ["Quero", "um", "pad", "atmosférico", "e", "com", "longo", "decay", "."],
        ["Aplique", "um", "compressor", "com", "ratio", "alto", "para", "uma", "voz", "agressiva", "."]
    ],
    "ner_tags": [
        [label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["I-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["I-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["B-PARAMETRO"], label_to_id["O"]],
        [label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["O"], label_to_id["B-PARAMETRO"], label_to_id["B-QUALIDADE"], label_to_id["O"], label_to_id["O"], label_to_id["B-QUALIDADE"], label_to_id["O"]]
    ]
}

# --- Importante: salve este 'data' em um arquivo JSON para uso real ---
# Você pode fazer isso para ter um arquivo de dataset separado
with open("musical_ner_dataset.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("Dataset de exemplo criado e salvo como 'musical_ner_dataset.json'")
print(f"Número de exemplos no dataset: {len(data['tokens'])}")
print("\nPrimeiro exemplo:")
print("Tokens:", data["tokens"][0])
print("NER Tags:", [id_to_label[tag_id] for tag_id in data["ner_tags"][0]])

Dataset de exemplo criado e salvo como 'musical_ner_dataset.json'
Número de exemplos no dataset: 20

Primeiro exemplo:
Tokens: ['Coloque', 'um', 'reverb', 'bem', 'espacial', 'na', 'voz', '.']
NER Tags: ['O', 'O', 'B-PARAMETRO', 'O', 'B-QUALIDADE', 'O', 'O', 'O']


In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import Dataset, Features, Sequence, ClassLabel, Value
from seqeval.metrics import classification_report
import torch
import numpy as np
import json

# 1. Carregar o Tokenizador e o Modelo Base TinyBERT
model_name = "google/bert_uncased_L-2_H-128_A-2" # Exemplo de TinyBERT
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Exemplo simplificado de labels para NER (precisa ser definido com base nos seus dados)
# Por exemplo, se você tem 'PARAMETRO' e 'QUALIDADE'
# 'O' = Outro
# 'B-' = Início de uma entidade
# 'I-' = Interior de uma entidade
# DEFINA SEUS LABELS AQUI, COM BASE NAS ENTIDADES QUE VOCÊ QUER EXTRAIR
raw_labels = ["O", "B-PARAMETRO", "I-PARAMETRO", "B-QUALIDADE", "I-QUALIDADE"]
label_to_id = {label: i for i, label in enumerate(raw_labels)}
id_to_label = {i: label for i, label in enumerate(raw_labels)}

# AGORA O MODELO PODE SER INSTANCIADO COM num_labels CORRETO
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(raw_labels))

# Se você tiver GPU, mova o modelo para a GPU (recomendado para treinamento)
if torch.cuda.is_available():
    model.to("cuda")

# 2. Preparar seu Dataset (NER)
# Seus dados viriam de um arquivo JSON, CSV, etc., rotulados com NER.
# Exemplo de como seus dados poderiam ser estruturados após o pré-processamento/rotulagem
# Isso é o formato esperado pelo Hugging Face para NER
with open("musical_ner_dataset.json", "r", encoding="utf-8") as f:
    raw_data_from_json = json.load(f)

# Criar o Dataset com o esquema correto de features
features = Features({
    'tokens': Sequence(Value('string')),
    'ner_tags': Sequence(ClassLabel(names=raw_labels))
})
dataset = Dataset.from_dict(raw_data_from_json, features=features)

# Função de tokenização e alinhamento de rótulos para NER
def tokenize_and_align_labels(examples):
    # Definindo max_length, por exemplo, 128
    # O TinyBERT (L-2_H-128_A-2) tem uma dimensão escondida de 128, então isso é razoável.
    # Se suas frases forem muito maiores, pode precisar de um TinyBERT maior ou truncar mais.
    max_length = 128
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=max_length
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Token especial (CLS, SEP, PAD)
            if word_idx is None:
                label_ids.append(-100) # Ignora esses tokens na perda
            # Adição da verificação: Apenas processa se word_idx estiver dentro dos limites de 'label'
            elif word_idx < len(label): # <-- ADIÇÃO CHAVE AQUI
                if word_idx != previous_word_idx:
                    # É o primeiro token de uma nova palavra
                    label_ids.append(label[word_idx])
                else:
                    # Se não é o primeiro token da palavra, mas faz parte dela
                    current_label_name = id_to_label[label[word_idx]]
                    if current_label_name.startswith("B-"):
                        label_ids.append(label_to_id[current_label_name.replace("B-", "I-")])
                    else:
                        label_ids.append(label[word_idx])
            else:
                # Caso word_idx seja maior ou igual ao tamanho de 'label', significa que
                # a palavra original correspondente foi truncada. Ignoramos o label.
                label_ids.append(-100) # Ignorar na perda

            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Dividir dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# 3. Definir Métricas de Avaliação
from seqeval.metrics import classification_report

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2) # Para NER, argmax na dimensão dos labels

    true_labels = [[id_to_label[l] for l in label if l != -100] for label in labels]
    true_predictions = [[id_to_label[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    # Debugging: Imprima o relatório para ver o que ele contém
    print("True Labels (Sample):", true_labels[:2])
    print("True Predictions (Sample):", true_predictions[:2])

    try:
        # Tenta gerar o relatório. Se não houver entidades, pode lançar um erro interno ou retornar vazio.
        report = classification_report(true_labels, true_predictions, output_dict=True)
    except ValueError as e:
        # Captura erros internos da seqeval, que podem ocorrer se input for inválido
        print(f"Warning: classification_report raised an error: {e}")
        # Retorna métricas padrão zero se houver erro
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0, "accuracy": 0.0}

    # Verifica se as chaves "overall_precision" etc. existem antes de acessá-las
    # Isso cobre o caso em que não há entidades a serem avaliadas
    if "overall_precision" in report:
        return {
            "precision": report["overall_precision"],
            "recall": report["overall_recall"],
            "f1": report["overall_f1"],
            "accuracy": report["overall_accuracy"],
        }
    else:
        # Se as chaves overall não estão presentes, significa que não havia entidades para avaliar
        print("Warning: 'overall_precision' not found in classification_report. Likely no entities in batch.")
        # Retorna métricas padrão zero para evitar KeyErrors
        return {"precision": 0.0, "recall": 0.0, "f1": 0.0, "accuracy": 0.0}

# 4. Configurar e Treinar o Trainer
training_args = TrainingArguments(
    output_dir="./tinybert_musical_ner_results",
    num_train_epochs=15,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./tinybert_musical_ner_logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1", # F1-score é uma boa métrica para NER
    report_to="none" # Desabilitar integração com outras ferramentas de log se não precisar
)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer, # Mantenha o tokenizer aqui por enquanto devido ao FutureWarning, ou mude para processing_class
    data_collator=data_collator, # <-- ADICIONE ISSO AQUI
    compute_metrics=compute_metrics
)

trainer.train()

# 5. Salvar o Modelo e o Tokenizador para Rust
trainer.save_model("./final_tinybert_musical_ner_model")
tokenizer.save_pretrained("./final_tinybert_musical_ner_model")

# E a lista de labels para Rust (importante para mapear os IDs de volta para strings)
import json
with open("./final_tinybert_musical_ner_model/labels.json", "w") as f:
    json.dump({"id_to_label": id_to_label, "label_to_id": label_to_id}, f)

# Exporte para ONNX (muito importante para Rust)
# O mesmo processo de exportação ONNX que mostrei antes, mas para AutoModelForTokenClassification
# Garanta que a entrada dummy seja adequada para token classification

# Exemplo de exportação ONNX para AutoModelForTokenClassification
dummy_input_ids = torch.randint(0, tokenizer.vocab_size, (1, 128)) # Batch size 1, seq_len 128
dummy_attention_mask = torch.ones(1, 128, dtype=torch.long)
dummy_token_type_ids = torch.zeros(1, 128, dtype=torch.long)

# Mover para GPU se o modelo estiver na GPU
if torch.cuda.is_available():
    dummy_input_ids = dummy_input_ids.to("cuda")
    dummy_attention_mask = dummy_attention_mask.to("cuda")
    dummy_token_type_ids = dummy_token_type_ids.to("cuda")

torch.onnx.export(
    model,
    (dummy_input_ids, dummy_attention_mask, dummy_token_type_ids),
    "tinybert_musical_ner_model.onnx", # Nome do arquivo ONNX
    input_names=["input_ids", "attention_mask", "token_type_ids"],
    output_names=["logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "token_type_ids": {0: "batch_size", 1: "sequence_length"},
        "logits": {0: "batch_size", 1: "sequence_length"} # Para TokenClassification, logits tbm tem sequence_length
    },
    opset_version=14, # Ou uma versão compatível com sua ferramenta ONNX em Rust
)

print(f"Total examples in dataset: {len(dataset)}")
print(f"Total examples in training set: {len(train_dataset)}")
print(f"Total examples in validation set: {len(eval_dataset)}")

# Count entities in your dataset (optional, but very helpful)
entity_counts = {label: 0 for label in raw_labels}
for example in dataset["ner_tags"]:
    for tag_id in example:
        if tag_id != -100: # Exclude padding/special tokens
            entity_counts[id_to_label[tag_id]] += 1
print("Entity counts in dataset:", entity_counts)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

<ipython-input-9-d61c3afe8dcb>:150: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,1.464088,0.000000,0.000000,0.000000,0.000000
2,No log,1.422441,0.000000,0.000000,0.000000,0.000000
3,No log,1.388370,0.000000,0.000000,0.000000,0.000000
4,No log,1.360703,0.000000,0.000000,0.000000,0.000000
5,1.363500,1.338613,0.000000,0.000000,0.000000,0.000000
6,1.363500,1.320733,0.000000,0.000000,0.000000,0.000000
7,1.363500,1.306032,0.000000,0.000000,0.000000,0.000000
8,1.363500,1.294254,0.000000,0.000000,0.000000,0.000000
9,1.363500,1.284783,0.000000,0.000000,0.000000,0.000000
10,1.208600,1.277246,0.000000,0.000000,0.000000,0.000000


True Labels (Sample): [['O', 'B-PARAMETRO', 'I-PARAMETRO', 'I-PARAMETRO', 'O', 'B-PARAMETRO', 'B-QUALIDADE', 'I-QUALIDADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-QUALIDADE', 'I-QUALIDADE', 'O'], ['O', 'B-PARAMETRO', 'O', 'O', 'O', 'O', 'B-QUALIDADE', 'I-QUALIDADE', 'O', 'O', 'B-PARAMETRO', 'B-QUALIDADE', 'I-QUALIDADE', 'O']]
True Predictions (Sample): [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'I-QUALIDADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


...

True Labels (Sample): [['O', 'B-PARAMETRO', 'I-PARAMETRO', 'I-PARAMETRO', 'O', 'B-PARAMETRO', 'B-QUALIDADE', 'I-QUALIDADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-QUALIDADE', 'I-QUALIDADE', 'O'], ['O', 'B-PARAMETRO', 'O', 'O', 'O', 'O', 'B-QUALIDADE', 'I-QUALIDADE', 'O', 'O', 'B-PARAMETRO', 'B-QUALIDADE', 'I-QUALIDADE', 'O']]
True Predictions (Sample): [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


True Labels (Sample): [['O', 'B-PARAMETRO', 'I-PARAMETRO', 'I-PARAMETRO', 'O', 'B-PARAMETRO', 'B-QUALIDADE', 'I-QUALIDADE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-QUALIDADE', 'I-QUALIDADE', 'O'], ['O', 'B-PARAMETRO', 'O', 'O', 'O', 'O', 'B-QUALIDADE', 'I-QUALIDADE', 'O', 'O', 'B-PARAMETRO', 'B-QUALIDADE', 'I-QUALIDADE', 'O']]
True Predictions (Sample): [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Total examples in dataset: 20
Total examples in training set: 16
Total examples in validation set: 4
Entity counts in dataset: {'O': 146, 'B-PARAMETRO': 23, 'I-PARAMETRO': 1, 'B-QUALIDADE': 30, 'I-QUALIDADE': 1}
